In [2]:
import pyodbc
import pandas as pd
import numpy as np
from sodapy import Socrata

## Secop2 Completo

In [12]:
def iniciarSesion():
    server = 'tcp:miadgpproyecto.database.windows.net' 
    database = 'DB_GP_PROYECTO' 
    username = 'admin_miad' 
    password = 'Abc123456789' 
    
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    
    return cnxn, cursor

In [4]:
def leerDatos(query):
    client = Socrata("www.datos.gov.co", None)
    results = client.get("p6dx-8zbt", query=query)
    results_df = pd.DataFrame.from_records(results)
    
    return results_df

In [5]:
def borrarSecop2Completo():
    cnxn.execute("""TRUNCATE TABLE Test_Secop2_Completo""")
    cnxn.commit()

In [15]:
def cargueSecop2Completo(cnxn, cursor, df_temp):
    print('Proceso iniciado')
    
    for index, row in df_temp.iterrows():
        cursor.execute("""INSERT INTO Test_Secop2_Completo ([Entidad], [Nit Entidad], [Departamento Entidad], [Ciudad Entidad],
       [OrdenEntidad], [Entidad Centralizada], [ID del Proceso],
       [Referencia del Proceso], [PCI], [ID del Portafolio],
       [Nombre del Procedimiento], [Fase],
       [Fecha de Publicacion del Proceso], [Fecha de Ultima Publicación],
       [Fecha de Publicacion (Manifestacion de Interes)],
       [Fecha de Publicacion (Fase Borrador)],
       [Fecha de Publicacion (Fase Seleccion)], [Precio Base],
       [Modalidad de Contratacion], [Justificación Modalidad de Contratación],
       [Duracion], [Unidad de Duracion], [Fecha de Recepcion de Respuestas],
       [Fecha de Apertura de Respuesta], [Fecha de Apertura Efectiva],
       [Ciudad de la Unidad de Contratación],
       [Nombre de la Unidad de Contratación], [Proveedores Invitados],
       [Proveedores con Invitacion Directa],
       [Visualizaciones del Procedimiento],
       [Proveedores que Manifestaron Interes], [Respuestas al Procedimiento],
       [Respuestas Externas], [Conteo de Respuestas a Ofertas],
       [Proveedores Unicos con Respuestas], [Numero de Lotes],
       [Estado del Procedimiento], [ID Estado del Procedimiento], [Adjudicado],
       [ID Adjudicacion], [CodigoProveedor], [Departamento Proveedor],
       [Ciudad Proveedor], [Fecha Adjudicacion], [Valor Total Adjudicacion],
       [Nombre del Adjudicador], [Nombre del Proveedor Adjudicado],
       [NIT del Proveedor Adjudicado], [Codigo Principal de Categoria],
       [Estado de Apertura del Proceso], [Tipo de Contrato],
       [Subtipo de Contrato], [URLProceso],
       [Codigo Entidad], [Estado Resumen]) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""", 
                      str(row.entidad), str(row.nit_entidad), str(row.departamento_entidad), str(row.ciudad_entidad),
       str(row.ordenentidad), str(row.codigo_pci), str(row.id_del_proceso),
       str(row.referencia_del_proceso), str(row.ppi), str(row.id_del_portafolio),
       str(row.nombre_del_procedimiento), str(row.fase),
       str(row.fecha_de_publicacion_del), str(row.fecha_de_ultima_publicaci),
       str(row.fecha_de_publicacion),
       str(row.fecha_de_publicacion_fase_2),
       str(row.fecha_de_publicacion_fase_3), str(row.precio_base),
       str(row.modalidad_de_contratacion), str(row.justificaci_n_modalidad_de), str(row.duracion),
       str(row.unidad_de_duracion), str(row.fecha_de_recepcion_de),
       str(row.fecha_de_apertura_de_respuesta), str(row.fecha_de_apertura_efectiva),
       str(row.ciudad_de_la_unidad_de), str(row.nombre_de_la_unidad_de), str(row.proveedores_invitados),
       str(row.proveedores_con_invitacion),
       str(row.visualizaciones_del), 
       str(row.proveedores_que_manifestaron), str(row.respuestas_al_procedimiento), 
       str(row.respuestas_externas), str(row.conteo_de_respuestas_a_ofertas),
       str(row.proveedores_unicos_con), str(row.numero_de_lotes),
       str(row.estado_del_procedimiento), str(row.id_estado_del_procedimiento), str(row.adjudicado), 
       str(row.id_adjudicacion), str(row.codigoproveedor), str(row.departamento_proveedor), 
       str(row.ciudad_proveedor), str(row.fecha_adjudicacion), str(row.valor_total_adjudicacion),
       str(row.nombre_del_adjudicador), str(row.nombre_del_proveedor),
       str(row.nit_del_proveedor_adjudicado), str(row.codigo_principal_de_categoria),
       str(row.estado_de_apertura_del_proceso), str(row.tipo_de_contrato),
       str(row.subtipo_de_contrato), str(row.urlproceso),
       str(row.codigo_entidad), str(row.estadoresumen))
    cnxn.commit()
    
    print('Datos cargados')

In [24]:
def cerrarCursor(cursor):
    try:
        cursor.close()
    except:
        pass

In [ ]:
# 1. Se inicia la conexion con la base de datos
cnxn, cursor = iniciarSesion()
print("Sesion iniciada")

query = "SELECT * WHERE tipo_de_contrato LIKE 'Obra' LIMIT 500000"
# 2. Se cargan los datos desde el API
results_df = leerDatos(query)
print("Datos descargados")

# 3. Se limipia la tabla para actualizar con los nuevos datos
borrarSecop2Completo()
cerrarCursor(cursor)
print("Base limpia")

# 4. Se cargan los datos completos a la base SQL
k = 0
total = 0
while total < results_df.shape[0]:
    cnxn, cursor = iniciarSesion()
    df_temp = results_df[k*1000:min((k+1)*1000, results_df.shape[0])]
    print(k)
    cargueSecop2Completo(cnxn, cursor, df_temp)
    print("Carga completa")
    k += 1
    total += df_temp.shape[0]
    print(total)
    cerrarCursor(cursor)

# 5. Se cierra el cursor
# cerrarCursor(cursor)
print("Proceso finalizado")

### Prueba de lectura

In [27]:
sql = """
SELECT *
FROM Test_Secop2_Completo
"""

In [28]:
data = pd.read_sql(sql,cnxn)

In [21]:
data.head()

,Entidad,Nit Entidad,Departamento Entidad,Ciudad Entidad,OrdenEntidad,Entidad Centralizada,ID del Proceso,Referencia del Proceso,PCI,ID del Portafolio,...,Nombre del Proveedor Adjudicado,NIT del Proveedor Adjudicado,Codigo Principal de Categoria,Estado de Apertura del Proceso,Tipo de Contrato,Subtipo de Contrato,Categorias Adicionales,URLProceso,Codigo Entidad,Estado Resumen
0,SUPERINTENDENCIA DE SOCIEDADES,899999086,No Definido,No Definido,Nacional,Si,CO1.REQ.107802,56 DE 2016,35-02-00,CO1.BDOS.105401,...,Construcciones AMG SAS,No Definido,V1.72121103,Cerrado,Obra,No Especificado,None,{'url': 'https://community.secop.gov.co/Public...,700326093,Adjudicado
1,INSTITUTO NACIONAL DE VIAS,800215807,Distrito Capital de Bogotá,Bogotá,Nacional,No,CO1.REQ.419945,SA-MC-DT-OCA-004-2018 (Manifestación de interé...,24-02-00,CO1.BDOS.394474,...,No Adjudicado,No Adjudicado,V1.72141000,Cerrado,Obra,No Especificado,None,{'url': 'https://community.secop.gov.co/Public...,700676059,Manifestación de interés (Menor Cuantía)
2,INSTITUTO DE INFRAESTRUCTURA Y CONCESIONES DE ...,900258711,No Definido,No Definido,Territorial,Si,CO1.REQ.3384622,ICCU-CTO-293 DE 2022,ND,CO1.BDOS.3297326,...,No Adjudicado,No Adjudicado,V1.95111600,Abierto,Obra,No Especificado,None,{'url': 'https://community.secop.gov.co/Public...,702046087,Adjudicado
3,MUNICIPIO DE EL BAGRE,890984221,No Definido,No Definido,Territorial,Si,CO1.REQ.3561961,CONV-256-2022,ND,CO1.BDOS.3470514,...,No Adjudicado,No Adjudicado,V1.72153702,Abierto,Obra,No Especificado,None,{'url': 'https://community.secop.gov.co/Public...,702797036,Adjudicado
4,INSTITUTO NACIONAL DE VIAS,800215807,Distrito Capital de Bogotá,Bogotá,Nacional,No,CO1.REQ.444075,SA-MC-DT-BOY-006-2018,24-02-00,CO1.BDOS.429645,...,No Adjudicado,No Adjudicado,V1.72141000,Cerrado,Obra,No Especificado,None,{'url': 'https://community.secop.gov.co/Public...,700676059,Presentación de observaciones


In [29]:
data.shape

(34756, 59)